---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [41]:
import pandas as pd
import datetime
pd.options.display.max_rows = 999

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [42]:
df = pd.DataFrame(doc, columns=['raw'])

In [43]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
              'August', 'September', 'October', 'November', 'December']
months_short = [x[0:3] for x in months]
months.extend(months_short)
regex_months = "|".join(months)

regex1 = "\d{0,2}[/-]*\d{0,2}[/-](?:[1-2]\d{3}|[6-9][0-9])" #03/25/93 or 4-13-82 or 9/1980. matches any year 60-99
regex2 = "[\d\d]*\s*(?:{}),*\s\d\d\d\d".format(regex_months)   # 09 Sep 1989 or Feb 1978
regex3 = "(?:{})\.*\s\d\d,*\s\d\d\d\d".format(regex_months)            #July 26, 1978 or Jan 24 1986
regex4 = "[1|2]\d\d\d"                        # 1998 or 2015
regex = "|".join([regex1, regex2, regex3, regex4 ])

df['clean'] = df.raw.str.findall(regex)

In [44]:
df.head(20)

,raw,clean
0,03/25/93 Total time of visit (in minutes):\n,[03/25/93]
1,6/18/85 Primary Care Doctor:\n,[6/18/85]
2,sshe plans to move as of 7/8/71 In-Home Servic...,[7/8/71]
3,7 on 9/27/75 Audit C Score Current:\n,[9/27/75]
4,2/6/96 sleep studyPain Treatment Pain Level (N...,[2/6/96]
5,.Per 7/06/79 Movement D/O note:\n,[7/06/79]
6,"4, 5/18/78 Patient's thoughts about current su...",[5/18/78]
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,[10/24/89]
8,3/7/86 SOS-10 Total Score:\n,[3/7/86]
9,(4/10/71)Score-1Audit C Score Current:\n,[4/10/71]


In [45]:
df.tail(20)

,raw,clean
480,"1. Esophageal cancer, dx: 2013, on FOLFOX with...",[2013]
481,y1974 (all)\n,[1974]
482,h/o restraining order by sister/mother in 1990...,[1990]
483,sTexas Medical Center; Oklahoma for 2 weeks; 1...,[1995]
484,Death of former partner in 2004 by overdose as...,[2004]
485,"Was ""average"" student. ""I didn't have too man...",[1987]
486,Contemplating jumping off building - 1973 - di...,[1973]
487,appendectomy s/p delivery 1992 Prior relevant ...,[1992]
488,tProblems renal cell cancer : s/p nephrectomy ...,[1977]
489,"ran own business for 35 years, sold in 1985\n",[1985]


In [46]:
df['date'] = df.clean.apply(lambda x: x[0]).str.strip().str.strip("-/")
df.loc[271, 'date'] = df.loc[271, 'clean'][2]
df['date'] = df['date'].str.replace("-", "/").str.replace(",|\.", " ").str.strip()


In [47]:
# Now I need to break down each date into DAY MONTH YEAR to align it in one format later
regex_year = "\d\d\d\d|\d{2,4}$"
df['year'] = df.date.str.findall(regex_year)
df['year'] = df['year'].apply(lambda x: x[0])

In [48]:
df.year = df.year.apply(lambda x: "19" + x if len(x) == 2 else x)

In [49]:
df.head(10)

,raw,clean,date,year
0,03/25/93 Total time of visit (in minutes):\n,[03/25/93],03/25/93,1993
1,6/18/85 Primary Care Doctor:\n,[6/18/85],6/18/85,1985
2,sshe plans to move as of 7/8/71 In-Home Servic...,[7/8/71],7/8/71,1971
3,7 on 9/27/75 Audit C Score Current:\n,[9/27/75],9/27/75,1975
4,2/6/96 sleep studyPain Treatment Pain Level (N...,[2/6/96],2/6/96,1996
5,.Per 7/06/79 Movement D/O note:\n,[7/06/79],7/06/79,1979
6,"4, 5/18/78 Patient's thoughts about current su...",[5/18/78],5/18/78,1978
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,[10/24/89],10/24/89,1989
8,3/7/86 SOS-10 Total Score:\n,[3/7/86],3/7/86,1986
9,(4/10/71)Score-1Audit C Score Current:\n,[4/10/71],4/10/71,1971


In [50]:
regex_month = "^\d{0,2}[/-]"
regex_month1 = "{}".format(regex_months)
regex_final = "|".join([regex_month, regex_month1])
df['month'] = df.date.str.findall(regex_final).apply(lambda x: x[0] if len(x) > 0 else 1).str.strip("/-")
df.month = df.month.str.slice(0,3)
df.month.fillna('1', inplace=True)

In [51]:
regex_months

'January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec'

In [52]:
month_dict = {'Jan' : 1,
             'Feb' : 2,
             'Mar' : 3,
             'Apr' : 4,
             'May' : 5,
             'Jun' : 6,
             'Jul' : 7,
             'Aug' : 8,
             'Sep' : 9,
             'Oct' : 10,
             'Nov' : 11,
             'Dec' : 12}

mask = df.month.str.contains(regex_months)
df.loc[mask, 'month'] = df.loc[mask, 'month'].map(month_dict)

In [53]:
regex_day = "\d[/](\d{0,2})[/]|(\d{0,2})\s[A-Z]"
df['day'] = df.date.str.findall(regex_day)

In [54]:
df.head(10)
#So this looks a bit ugly cos i have two caputuring groups. 
# on the other hand the regex is a one liner which is nice...

,raw,clean,date,year,month,day
0,03/25/93 Total time of visit (in minutes):\n,[03/25/93],03/25/93,1993,03,"[(25, )]"
1,6/18/85 Primary Care Doctor:\n,[6/18/85],6/18/85,1985,6,"[(18, )]"
2,sshe plans to move as of 7/8/71 In-Home Servic...,[7/8/71],7/8/71,1971,7,"[(8, )]"
3,7 on 9/27/75 Audit C Score Current:\n,[9/27/75],9/27/75,1975,9,"[(27, )]"
4,2/6/96 sleep studyPain Treatment Pain Level (N...,[2/6/96],2/6/96,1996,2,"[(6, )]"
5,.Per 7/06/79 Movement D/O note:\n,[7/06/79],7/06/79,1979,7,"[(06, )]"
6,"4, 5/18/78 Patient's thoughts about current su...",[5/18/78],5/18/78,1978,5,"[(18, )]"
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,[10/24/89],10/24/89,1989,10,"[(24, )]"
8,3/7/86 SOS-10 Total Score:\n,[3/7/86],3/7/86,1986,3,"[(7, )]"
9,(4/10/71)Score-1Audit C Score Current:\n,[4/10/71],4/10/71,1971,4,"[(10, )]"


In [55]:
df['day'] = df['day'].apply(lambda x: x[0] if len(x) > 0 else 1)
df.day = df.day.apply(lambda x : list(filter(bool, x)) if type(x) is tuple else x)
df.day = df.day.apply(lambda x: x[0] if (( type(x) is list) and (len(x) > 0)) else x)

In [56]:
df['final_date'] = df.apply(lambda row: datetime.date(int(row['year']), int(row['month']), int(row['day'])) , axis=1)
df.sort_values(by="final_date", inplace=True)
result = pd.Series(df.index)

In [68]:
result.head(20)

0       9
1      84
2       2
3      53
4      28
5     474
6     153
7      13
8     129
9      98
10    225
11    111
12     31
13    171
14    191
15    486
16    335
17    415
18     36
19    323
dtype: int64

In [69]:
# DONE

In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here